# Generalized Advantage Estimation

## 목적

- 현재 policy gradient method의 목적은 누적 discount reward를 optimization하는 것이다. 하지만 학습할 때 많은 양의 sample이 필요하고 sample data가 nonstationarity임에도 stable한 학습이 어렵다.
- 따라서, \\(TD(\lambda)\\)와 같은 advantage function의 exponentially-weighted estimator를 사용하여 policy gradient estimate의 variance를 줄인다. variance를 줄이게 되면 stable하게 학습을 할 수 있다.
- 또한, policy와 value function에 대한 Trust Region optimization(앞선 TRPO에서 소개되었다.) 사용

## 요약

- 따라서, advantage function의 good estimate을 얻는 variance reduction에 대해서 알아보자.
  - 위 문제를 해결하기 위해서, Generalized advantage estimator를 사용했고, 이것은 bias-variance tradeoff를 조절하는 두개의 parameter \\(\gamma, \lambda\\)가 있다.
  - 그리고, Trust region policy optimization과 value function을 optimize하는 trust region 알고리즘의 아이디어를 어떻게 합치는지 증명하였다.

- REINFORCE, expected total returns의 gradient에 대한 unbiased estimate을 REINFORCE라 한다. 하지만 하나의 에피소드가 끝날 때 update를 진행하는 방식이므로 높은 variance를 가지게 된다.
- Actor-critic, 하나의 value function을 사용한다. 그리고 bias하고 낮은 variance를 가진다. 만약 높은 variance를 가진다면 sampling을 더 하면 되는데 bias한 경우는 치명적이다. bias한 경우는 수렴이 되지 않거나 local optimum이 아닌 poor solution에 수렴하도록 만든다.

## 준비

- 일단 policy optimization의 undicounted formulation을 가정한다. (원래는 discount를 붙여주지만 여기선 붙이지 않는 것을 가정한다.)
- initial state \\(s_0\\)는 distribution \\(\rho_0\\)로 부터 sampling된 것이다.
- 하나의 trajectory(\\(s_0, a_0, s_1, a_1, ...)\\))는 terminal state에 도달할 때 까지 policy \\(a_t \sim \pi(a_t|s_t)\\)에 따라서 action을 sampling하고, dynamics \\(s_{t+1} \sim P(s_{t + 1}|s_t, a_t)\\)에 따라서 state를 sampling함으로써 생성된다.
- reward, \\(r_t = r(s_t, a_t, s_{t+1})\\)은 매 time step마다 받아진다.
- 우리의 목표는 모든 policy에 대해서 finite하다고 가정됨으로서 expected total reward \\(\sum^\infty_{t=0} r_t\\)를 maximize하는 것이다.


- 앞서 가정했듯이 undiscounted formulation이고, 따라서 \\(\gamma\\)를 discount의 parameter로 사용하는 것이 아니라 bias-variance tradeoff를 조절하는 parameter로 사용한다.

- policy gradient method는 gradient \\(g := \triangledown_\theta \mathbb{E}[\sum^\infty_{t=0} r_t] \\)를 반복적으로 estimate함으로써 expected total reward를 maximize하는 것인데, policy gradient에는 여러가지 표현들이 있다.

$$ g = \mathbb{E}[\sum^\infty_{t=0} \Phi_t \triangledown_\theta log \pi_\theta(a_t|s_t)] $$
- 위 식에서 \\(\Phi_t\\)는 아래식 중 하나가 될 수 있다.
  1. \\(\sum^\infty_{t=0} r_t\\), trajectory의 total reward
  2. \\(\sum^\infty_{t'=t} r'_t\\), action \\(a_t\\)후의 reward
  3. \\(\sum^\infty_{t'=t} r'_t - b(s_t)\\), 위 2번 식의 baselined된 reward
  4. \\(Q^\pi(s_t, a_t)\\), state-action value function
  5. \\(A^\pi(s_t, a_t)\\), advantage function, advantage를 빼줘서 높은 variance를 보정한다.
  6. \\(r_t + V^\pi(s_{t+1}) - V^\pi(s_t)\\), TD error
- 위 식들중 4, 5, 6은 아래 수식을 사용한다.
  - \\(V^\pi(s_t) := \mathbb{E}_{s_{t+1}:\infty, a_t:\infty}[\sum^\infty_{l=0} r_{t+1}]\\)
  - \\(Q^\pi(s_t, a_t) := \mathbb{E}_{s_{t+1}:\infty, a_{t+1}:\infty}[\sum^\infty_{l=0} r_{t+l}]\\)
  - \\(A^pi(s_t, a_t) := Q^\pi(s_t, a_t) - V^\pi(s_t)\\)
- notation
  - \\(a:b\\)의 의미는, \\((a, a + 1, ... , b)\\)이다.

# 참조

[GAE 설명 블로그](http://dongminlee.tistory.com/12)